In [1]:
from main import FileExtractPhase

#spark = PysparkManager().CreateSparkSession()

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]") \
                .appName("PySparkShell") \
                .config("spark.jars", "postgresql-42.6.0.jar") \
                .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/09/15 12:34:31 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.22.92.204 instead (on interface eth0)
23/09/15 12:34:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/09/15 12:34:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [4]:
import configparser

config = configparser.ConfigParser(); config.read('config.ini')
database_url = config.get("DATABASE", "ConnectionUrl")

properties = {
    "driver": config.get("DATABASE", "driver"),
    "user": config.get("DATABASE", "ConnectionUser"),
    "password": config.get("DATABASE", "ConnectionPassword")
}

In [5]:
table_name = "warehouse.test"

df = spark.read.jdbc(database_url, table_name, properties=properties)

In [6]:
df = FileExtractPhase(spark, directory="Tripdata-Source")
#df.cache()

In [7]:
%%time
df.printSchema()
df.count()

root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- start_lat: double (nullable = true)
 |-- start_lng: double (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lng: double (nullable = true)
 |-- member_casual: string (nullable = true)

CPU times: user 7.86 ms, sys: 0 ns, total: 7.86 ms
Wall time: 665 ms


170360

In [8]:
df.cache()

23/08/03 12:12:22 WARN CacheManager: Asked to cache already cached data.


DataFrame[ride_id: string, rideable_type: string, started_at: timestamp, ended_at: timestamp, start_station_name: string, start_station_id: string, end_station_name: string, end_station_id: string, start_lat: double, start_lng: double, end_lat: double, end_lng: double, member_casual: string]

In [8]:
data_schema = df.schema

for field in data_schema.fields:
    null_count = df.filter(df[field.name].isNull()).count(); break
if null_count == 0 and len(df.columns) == 13 and len(df.columns) == 13:
    print("Data quality check passed")
else:
    print("Data quality check failed")

column_count = len(df.columns)
column_count

Data quality check passed


13

In [13]:
# SQL Select query
df.createOrReplaceTempView("mytable")
df.count()
spark.sql("select * from mytable").show(10)
df.write.saveAsTable("mytable")

+----------------+-------------+-------------------+-------------------+------------------+----------------+--------------------+--------------+-----------------+------------------+------------------+------------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|start_station_name|start_station_id|    end_station_name|end_station_id|        start_lat|         start_lng|           end_lat|           end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+------------------+----------------+--------------------+--------------+-----------------+------------------+------------------+------------------+-------------+
|38F943EA06A4E54F|electric_bike|2023-03-25 09:20:57|2023-03-25 09:32:10|   Jersey & 6th St|           JC027|        Glenwood Ave|         JC094|40.72528910781132|-74.04557168483734| 40.72755146730012|-74.07106071710587|       member|
|C76EFC4C3B771E88|electric_bike|2023-03-15 07:55:41|2023-03-15 0

In [ ]:
from pyspark.sql.functions import col
#df = df.withColumn("started_at", col("started_at").cast("string "))
#df = df.withColumn("ended_at", col("ended_at").cast("string "))

df.limit(5).toPandas().head()


In [ ]:
spark.catalog.clearCache()

In [ ]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)
df.show()

In [14]:
spark.stop()